###### 참고 자료

###### 효율적인 SVM을 위한 Bagging
###### https://m.blog.naver.com/cjh226/221359032956
###### 2.3.6 결정 트리의 앙상블
###### https://tensorflow.blog/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/2-3-6-%EA%B2%B0%EC%A0%95-%ED%8A%B8%EB%A6%AC%EC%9D%98-%EC%95%99%EC%83%81%EB%B8%94/
###### 앙상블 학습과 랜덤 포레스트
###### https://hoony-gunputer.tistory.com/entry/%ED%95%B8%EC%A6%88%EC%98%A8-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-7%EA%B0%95-%EC%95%99%EC%83%81%EB%B8%94-%ED%95%99%EC%8A%B5%EA%B3%BC-%EB%9E%9C%EB%8D%A4-%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8ensemble-RandomForest
###### 랜덤 포레스트
###### https://m.blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220979751089&proxyReferer=https:%2F%2Fwww.google.com%2F
###### 반복 교차 검증
###### https://tensorflow.blog/tag/gridsearchcv/
###### 3.3 scikit-learn의 전처리 기능
###### https://datascienceschool.net/view-notebook/f43be7d6515b48c0beb909826993c856/
###### 커널 서포트 벡터 머신
###### https://datascienceschool.net/view-notebook/69278a5de79449019ad1f51a614ef87c/
###### 2.3.7 커널 서포트 벡터 머신
###### https://tensorflow.blog/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/2-3-7-%EC%BB%A4%EB%84%90-%EC%84%9C%ED%8F%AC%ED%8A%B8-%EB%B2%A1%ED%84%B0-%EB%A8%B8%EC%8B%A0/
###### 머신러닝 - 3. 서포트 벡터 머신 (SVM) 실습
###### https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-3%EC%84%9C%ED%8F%AC%ED%8A%B8-%EB%B2%A1%ED%84%B0-%EB%A8%B8%EC%8B%A0-SVM-%EC%8B%A4%EC%8A%B5
###### 교차검증 (cross-validation)
###### https://homeproject.tistory.com/6
###### scikit-learn의 SVM을 통한 분류(Classification)
###### http://www.gisdeveloper.co.kr/?p=8180
###### 22. Python - 다중 분류
###### https://analysis-flood.tistory.com/77?category=725388

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn import svm
from sklearn.multiclass import OneVsOneClassifier

from sklearn.preprocessing import StandardScaler

In [2]:
# csv를 가져와 넘파이배열 형태의 데이터셋으로 가공하는 과정
import csv
f1=open('winequality-red.csv','r',encoding='UTF-8')
f2=open('winequality-white.csv','r',encoding='UTF-8')
red_wine_data = csv.reader(f1)
white_wine_data = csv.reader(f2)
raw1=[]# 가공 전 레드와인 데이터를 넣어둘 리스트
raw2=[]# 가공 전 화이트와인 데이터를 넣어둘 리스트

total_data=[]# 가공된 모든 와인 데이터를 넣어둘 리스트

features=[]# 특성 이름을 모아둘 리스트
feat=[]# csv에 "특성이름" 의 형태로 들어가 있어서 ""을 제거하기 전 특성 이름 리스트
target=[]# 정답인 등급을 모아둘 리스트

for r in red_wine_data:
    r=r[0].split(';')
    r.insert(0,'1')#red가 1
    raw1.append(r)

for w in white_wine_data:
    w=w[0].split(';')
    w.insert(0,'0')#white가 0
    raw2.append(w)

raw1[0].pop(0)# 와인의 색 정보로 위에서 넣어둔 1을 빼냄
feat=raw1[0]
for f in feat:
    s=f[1:len(f)-1]# ""을 제거하는 부분
    features.append(s)
features.insert(0,'color')

# csv의 첫번째 행이 특성 정보이므로 그것을 없애는 작업
raw1.pop(0)
raw2.pop(0)

for i1 in raw1:
    total_data.append(i1)

for i2 in raw2:
    total_data.append(i2)

for t in total_data:
    target.append(t.pop())


total_data=np.array(total_data)
total_data = total_data.astype(np.float64)# 실행했을때 밑의 경고문이 떠서 추가
"""
FutureWarning:
Beginning in version 0.22,
arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'.
It is recommended that you convert the array to a float dtype before using it in scikit-learn
"""

import copy

target_6=copy.deepcopy(target) # 1~5 / 6~10
      
for i in range(0,len(target_6)):
    if int(target_6[i])<5:
        target_6[i]='0'
    else:
       target_6[i]='1'
       
target_6=np.array(target_6)
target_6 = target_6.astype(np.float64)

f1.close()
f2.close()

#위에서 copy가 일어나므로 밑에서 처리
target=np.array(target)
target = target.astype(np.float64)

# 이진 분류

In [5]:
# 훈련 데이터와 테스트 데이터 이진분할
x_train, x_test, y_train, y_test = train_test_split(
    total_data, target_6, random_state=0)

In [6]:
scaler_train = StandardScaler()
scaler_train.fit(x_train)
X_train = scaler_train.transform(x_train)

scaler_test = StandardScaler()
scaler_test.fit(x_test)
X_test = scaler_test.transform(x_test)

### Linearly Separable Data without Noise

In [7]:
# make a classifier and fit on training data
clf_1 = svm.SVC(kernel='linear')

# Train classifier 
clf_1.fit(X_train, y_train)

# Make predictions on unseen test data
clf_1_predictions = clf_1.predict(X_test)
print("Accuracy: {}%".format(clf_1.score(X_test, y_test) * 100 ))

score_1_cv = cross_val_score(clf_1, total_data, target_6)
print('cross vaidation score : %s'%score_1_cv)
print(score_1_cv.mean())

skf=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
score_1_skf = cross_val_score(clf_1, total_data, target_6, cv=skf)
print('stratified k-fold cross vaidation score : %s'%score_1_skf)
print(score_1_skf.mean())

Accuracy: 96.0%
cross vaidation score : [0.96230769 0.96153846 0.96227868 0.96227868 0.96227868]
0.9621364363119559
stratified k-fold cross vaidation score : [0.96307692 0.96153846 0.96153846 0.96153846 0.96153846 0.96153846
 0.96153846 0.96302003 0.96302003 0.96302003]
0.9621367784757616


### Linearly Separable Data with Noise

In [9]:
# make a classifier and fit on training data
clf_2 = svm.SVC(kernel='linear', C=1)

# Train classifier 
clf_2.fit(X_train, y_train)

# Make predictions on unseen test data
clf_2_predictions = clf_2.predict(X_test)
print("Train Accuracy: {}%".format(clf_2.score(X_train, y_train) * 100 ))
print("Test Accuracy: {}%".format(clf_2.score(X_test, y_test) * 100 ))

# Parameter Grid
param_2_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Make grid search classifier
clf_2_grid = GridSearchCV(svm.SVC(), param_2_grid, verbose=1)

# Train the classifier
clf_2_grid.fit(X_train, y_train)

# clf = grid.best_estimator_()
print("Best Parameters:\n", clf_2_grid.best_params_)
print("Best Estimators:\n", clf_2_grid.best_estimator_)

print("Train Accuracy: {}%".format(clf_2_grid.score(X_train, y_train) * 100 ))
print("Test Accuracy: {}%".format(clf_2_grid.score(X_test, y_test) * 100 ))

Train Accuracy: 96.28489326765188%
Test Accuracy: 96.0%
Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Parameters:
 {'C': 0.001}
Best Estimators:
 SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Train Accuracy: 96.28489326765188%
Test Accuracy: 96.0%


[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    4.2s finished


In [10]:
# make a classifier and fit on training data
clf_2_bp = svm.SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Train classifier 
clf_2_bp.fit(X_train, y_train)

# Make predictions on unseen test data
clf_2_bppredictions = clf_2_bp.predict(X_test)
print("Train Accuracy: {}%".format(clf_2_bp.score(X_train, y_train) * 100 ))
print("Test Accuracy: {}%".format(clf_2_bp.score(X_test, y_test) * 100 ))

score_2_bp_cv = cross_val_score(clf_2_bp, total_data, target_6)
print('cross vaidation score : %s'%score_2_bp_cv)
print(score_2_bp_cv.mean())

skf=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
score_2_bp_skf = cross_val_score(clf_2_bp, total_data, target_6, cv=skf)
print('stratified k-fold cross vaidation score : %s'%score_2_bp_skf)
print(score_2_bp_skf.mean())

Train Accuracy: 96.28489326765188%
Test Accuracy: 96.0%
cross vaidation score : [0.96230769 0.96153846 0.96227868 0.96227868 0.96227868]
0.9621364363119559
stratified k-fold cross vaidation score : [0.96307692 0.96153846 0.96153846 0.96153846 0.96153846 0.96153846
 0.96153846 0.96302003 0.96302003 0.96302003]
0.9621367784757616


### NonLinearly Separable Data with Noise

In [11]:
# make a classifier
clf_3 = svm.SVC(C = 10.0, kernel='rbf', gamma=0.1)

# Train classifier
clf_3.fit(X_train, y_train)

# Make predictions on unseen test data
clf_3_predictions = clf_3.predict(X_test)
print("Train Accuracy: {}%".format(clf_3.score(X_train, y_train) * 100 ))
print("Test Accuracy: {}%".format(clf_3.score(X_test, y_test) * 100 ))

# Parameter Grid
param_3_grid = {'C': [1000, 100, 10, 1, 0.1, 0.01, 0.001], 
                'gamma': [1000, 100, 10, 1, 0.1, 0.01, 0.001]}

# Make grid search classifier
clf_3_grid = GridSearchCV(svm.SVC(), param_3_grid, verbose=1)

# Train the classifier
clf_3_grid.fit(X_train, y_train)

# clf = grid.best_estimator_()
print("Best Parameters:\n", clf_3_grid.best_params_)
print("Best Estimators:\n", clf_3_grid.best_estimator_)

Train Accuracy: 97.3111658456486%
Test Accuracy: 96.3076923076923%
Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:  1.7min finished


Best Parameters:
 {'C': 1000, 'gamma': 1000}
Best Estimators:
 SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1000, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [12]:
# make a classifier and fit on training data
clf_3_bp = svm.SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1000, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Train classifier 
clf_3_bp.fit(X_train, y_train)

# Make predictions on unseen test data
clf_3_bppredictions = clf_3_bp.predict(X_test)
print("Train Accuracy: {}%".format(clf_3_bp.score(X_train, y_train) * 100 ))
print("Test Accuracy: {}%".format(clf_3_bp.score(X_test, y_test) * 100 ))

score_3_bp_cv = cross_val_score(clf_3_bp, total_data, target_6)
print('cross vaidation score : %s'%score_3_bp_cv)
print(score_3_bp_cv.mean())

skf=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
score_3_bp_skf = cross_val_score(clf_3_bp, total_data, target_6, cv=skf)
print('stratified k-fold cross vaidation score : %s'%score_3_bp_skf)
print(score_3_bp_skf.mean())

Train Accuracy: 100.0%
Test Accuracy: 96.0%
cross vaidation score : [0.96230769 0.96153846 0.96227868 0.96227868 0.96227868]
0.9621364363119559
stratified k-fold cross vaidation score : [0.96615385 0.96461538 0.96615385 0.96307692 0.96461538 0.96307692
 0.96153846 0.96302003 0.96764253 0.96918336]
0.9649076686025838


# 다중분류

In [ ]:
# 훈련 데이터와 테스트 데이터 이진분할
x_train, x_test, y_train, y_test = train_test_split(
    total_data, target, random_state=0)

scaler_train = StandardScaler()
scaler_train.fit(x_train)
X_train = scaler_train.transform(x_train)

scaler_test = StandardScaler()
scaler_test.fit(x_test)
X_test = scaler_test.transform(x_test)

In [13]:
# make a classifier
clf = svm.SVC(C = 10.0, kernel='rbf', gamma=0.1)

# Train classifier
clf.fit(X_train, y_train)

# Make predictions on unseen test data
clf_predictions = clf.predict(X_test)
print("Train Accuracy: {}%".format(clf.score(X_train, y_train) * 100 ))
print("Test Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

# Parameter Grid
param_grid = {'C': [1000, 100, 10, 1, 0.1, 0.01, 0.001], 'gamma': [1000, 100, 10, 1, 0.1, 0.01, 0.001]}

# Make grid search classifier
clf_grid = GridSearchCV(svm.SVC(), param_grid, verbose=1)

# Train the classifier
clf_grid.fit(X_train, y_train)

# clf = grid.best_estimator_()
print("Best Parameters:\n", clf_grid.best_params_)
print("Best Estimators:\n", clf_grid.best_estimator_)

Train Accuracy: 97.3111658456486%
Test Accuracy: 96.3076923076923%
Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed:  1.7min finished


Best Parameters:
 {'C': 1000, 'gamma': 1000}
Best Estimators:
 SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1000, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [14]:
# make a classifier and fit on training data
clf_bp = svm.SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1000, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Train classifier 
clf_bp.fit(X_train, y_train)

# Make predictions on unseen test data
clf_bp_predictions = clf_bp.predict(X_test)
print("Train Accuracy: {}%".format(clf_bp.score(X_train, y_train) * 100 ))
print("Test Accuracy: {}%".format(clf_bp.score(X_test, y_test) * 100 ))

score_bp_cv = cross_val_score(clf_bp, total_data, target)
print('cross vaidation score : %s'%score_bp_cv)
print(score_bp_cv.mean())

skf=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
score_bp_skf = cross_val_score(clf_bp, total_data, target, cv=skf)
print('stratified k-fold cross vaidation score : %s'%score_bp_skf)
print(score_bp_skf.mean())

Train Accuracy: 100.0%
Test Accuracy: 96.0%
cross vaidation score : [0.43615385 0.43615385 0.43725943 0.43879908 0.43725943]
0.43712512583644225


C:\Users\SomeoneFromSomewhere\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


stratified k-fold cross vaidation score : [0.58923077 0.6        0.61230769 0.61538462 0.60923077 0.59076923
 0.61076923 0.61941448 0.62865948 0.59938367]
0.6075149934810952


In [15]:
skf_5=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score_bp_skf_5 = cross_val_score(clf_bp, total_data, target, cv=skf_5)
print('stratified k-fold cross vaidation score : %s'%score_bp_skf_5)
print(score_bp_skf_5.mean())

stratified k-fold cross vaidation score : [0.57230769 0.59615385 0.58352579 0.59276366 0.58891455]
0.586733108308166


# 랜덤 포레스트

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

ml = RandomForestClassifier(criterion='entropy',n_estimators=1000,
                           n_jobs=-1, random_state=1)

ml.fit(X_train, y_train)
y_pred = ml.predict(X_test)
print('총 테스트 개수:%d 오류개수:%d'%(len(y_test), (y_test != y_pred).sum()))
print('정확도: %.2f'%accuracy_score(y_test, y_pred))

'''
n_estimators=10
총 테스트 개수:1625 오류개수:60
정확도: 0.96

n_estimators=20
총 테스트 개수:1625 오류개수:62
정확도: 0.96

n_estimators=40
총 테스트 개수:1625 오류개수:64
정확도: 0.96

n_estimators=80
총 테스트 개수:1625 오류개수:62
정확도: 0.96

n_estimators=1000
총 테스트 개수:1625 오류개수:62
정확도: 0.96
'''

총 테스트 개수:1625 오류개수:62
정확도: 0.96


'\nn_estimators=10\n총 테스트 개수:1625 오류개수:60\n정확도: 0.96\n\nn_estimators=20\n총 테스트 개수:1625 오류개수:62\n정확도: 0.96\n\nn_estimators=40\n총 테스트 개수:1625 오류개수:64\n정확도: 0.96\n\nn_estimators=80\n총 테스트 개수:1625 오류개수:62\n정확도: 0.96\n'